In [163]:
import pandas as pd
import numpy as np
from datetime import datetime

In [164]:
# Function for Normalization
def normalize(series):
    min_val, max_val = series.min(), series.max()
    return (series - min_val) / (max_val - min_val) if max_val > min_val else 0

In [165]:
#global variables
current_date = datetime.now().strftime("%Y%m%d")
COUNTRY_SCORE_WEIGHTS = {'Exports': 0.57, 'Imports': 0.43, 'Gross_Deals': 0.0}
CATEGORY_SCORE_WEIGHTS = {'Exports': 0.57, 'Imports': 0.43, 'Gross_Deals': 0.0}


# Load the dataset
df = pd.read_csv("final_data/20241122one_row_one_country_data.csv")

In [166]:
# Compute country_metrics
exports = df[df['Country Role'] == 'Supplier'].groupby('Country')['SIPRI TIV for total order'].sum()
imports = df[df['Country Role'] == 'Recipient'].groupby('Country')['SIPRI TIV for total order'].sum().abs()
gross_deals = exports.abs() + imports

# Normalize country_metrics
country_metrics = pd.DataFrame({'Exports': exports, 'Imports': imports, 'Gross_Deals': gross_deals}).fillna(0)
country_metrics['Normalized_Exports'] = normalize(country_metrics['Exports'])
country_metrics['Normalized_Imports'] = normalize(country_metrics['Imports'])
country_metrics['Normalized_Gross'] = normalize(country_metrics['Gross_Deals'])

# Compute composite score

country_metrics['Country_Score'] = round(
    COUNTRY_SCORE_WEIGHTS['Exports'] * country_metrics['Normalized_Exports'] +
    COUNTRY_SCORE_WEIGHTS['Imports'] * country_metrics['Normalized_Imports'] +
    COUNTRY_SCORE_WEIGHTS['Gross_Deals'] * country_metrics['Normalized_Gross']
    , 5
) * 100

# Sort by score
country_metrics = country_metrics.sort_values(by='Country')
display(country_metrics)


,Exports,Imports,Gross_Deals,Normalized_Exports,Normalized_Imports,Normalized_Gross,Country_Score
Country,,,,,,,
ANC (South Africa)*,0.00,0.24,0.00,0.000000,0.000002,0.000000,0.000
Afghanistan,0.00,15079.26,0.00,0.000000,0.107573,0.000000,4.626
African Union**,0.00,88.58,0.00,0.000000,0.000632,0.000000,0.027
Albania,1.20,2218.94,2220.14,0.000002,0.015830,0.002795,0.681
Algeria,12.06,33663.90,33675.96,0.000016,0.240153,0.042398,10.328
...,...,...,...,...,...,...,...
Zimbabwe,5.01,850.92,855.93,0.000007,0.006070,0.001078,0.261
eSwatini,0.00,9.94,0.00,0.000000,0.000071,0.000000,0.003
unknown rebel group*,0.00,0.20,0.00,0.000000,0.000001,0.000000,0.000


In [184]:
# Initialize an empty list to store rows for the final DataFrame
rows = []

# Get the unique categories
categories = df['Category'].unique()

# Loop through each category to compute metrics
for category in categories:
    # Filter data for the current category
    category_data = df[df['Category'] == category]
    
    # Compute Exports, Imports, and Gross Deals for the current category
    exports = category_data[category_data['Country Role'] == 'Supplier'].groupby('Country')['SIPRI TIV for total order'].sum()
    imports = category_data[category_data['Country Role'] == 'Recipient'].groupby('Country')['SIPRI TIV for total order'].sum().abs()
    gross_deals = exports.abs() + imports
    
    # Normalize category_metrics
    category_metrics = pd.DataFrame({'Exports': exports, 'Imports': imports, 'Gross_Deals': gross_deals}).fillna(0)
    category_metrics['Normalized_Exports'] = (category_metrics['Exports'] - category_metrics['Exports'].min()) / (category_metrics['Exports'].max() - category_metrics['Exports'].min())
    category_metrics['Normalized_Imports'] = (category_metrics['Imports'] - category_metrics['Imports'].min()) / (category_metrics['Imports'].max() - category_metrics['Imports'].min())
    category_metrics['Normalized_Gross'] = (category_metrics['Gross_Deals'] - category_metrics['Gross_Deals'].min()) / (category_metrics['Gross_Deals'].max() - category_metrics['Gross_Deals'].min())
    
    # Define weights and calculate composite score
    category_metrics['Country_Score'] = round(
        CATEGORY_SCORE_WEIGHTS['Exports'] * category_metrics['Normalized_Exports'] +
        CATEGORY_SCORE_WEIGHTS['Imports'] * category_metrics['Normalized_Imports'] +
        CATEGORY_SCORE_WEIGHTS['Gross_Deals'] * category_metrics['Normalized_Gross']
        , 5
    ) * 100
    
    # Add rows for each country in the category
    for country, data in category_metrics.iterrows():
        rows.append({'Country': country, 'Metric Scope': 'Import', 'Metric Name': category, 'Value': data['Normalized_Imports']*CATEGORY_SCORE_WEIGHTS['Imports']*100})
        rows.append({'Country': country, 'Metric Scope': 'Export', 'Metric Name': category, 'Value': data['Normalized_Exports']*CATEGORY_SCORE_WEIGHTS['Exports']*100})
        rows.append({'Country': country, 'Metric Scope': 'Overall', 'Metric Name': category, 'Value': data['Country_Score']})

# Convert rows into a DataFrame
final_metrics_df = pd.DataFrame(rows)

# Display the final DataFrame
display(final_metrics_df)

,Country,Metric Scope,Metric Name,Value
0,Afghanistan,Import,Aircraft,3.161037
1,Afghanistan,Export,Aircraft,0.000000
2,Afghanistan,Overall,Aircraft,3.161000
3,Albania,Import,Aircraft,0.434435
4,Albania,Export,Aircraft,0.000000
...,...,...,...,...
4090,Zambia,Export,Naval Vessels,0.000000
4091,Zambia,Overall,Naval Vessels,0.020000
4092,unknown supplier(s),Import,Naval Vessels,0.000000
4093,unknown supplier(s),Export,Naval Vessels,0.018716


In [189]:
# Adding loop score copy of Aircraft for RADAR chart
aircraft_rows = final_metrics_df[final_metrics_df['Metric Name'] == 'Aircraft'].copy()
aircraft_rows['Metric Name'] = 'ZZ_loop_score_Aircraft'
final_metrics_df = pd.concat([final_metrics_df,aircraft_rows], ignore_index=True)
display(final_metrics_df)

,Country,Metric Scope,Metric Name,Value
0,Afghanistan,Import,Aircraft,3.161037
1,Afghanistan,Export,Aircraft,0.000000
2,Afghanistan,Overall,Aircraft,3.161000
3,Albania,Import,Aircraft,0.434435
4,Albania,Export,Aircraft,0.000000
...,...,...,...,...
4672,unknown recipient(s),Export,ZZ_loop_score_Aircraft,0.000000
4673,unknown recipient(s),Overall,ZZ_loop_score_Aircraft,0.004000
4674,unknown supplier(s),Import,ZZ_loop_score_Aircraft,0.000000
4675,unknown supplier(s),Export,ZZ_loop_score_Aircraft,0.053452


In [192]:
#Exporting the result to a csv file
final_metrics_df.to_csv('final_data/'+current_date+'_country_category_score_data.csv')

#Test Code Below

In [191]:
# Inner join based on 'Country'
result = pd.merge(final_metrics_df, country_metrics, on='Country', how='outer')
# result_with_loop = pd.concat([result, final_metrics_df['Aircraft'].rename('ZZ_loop_score')], axis=1)
display(result)

,Country,Metric Scope,Metric Name,Value,Exports,Imports,Gross_Deals,Normalized_Exports,Normalized_Imports,Normalized_Gross,Country_Score
0,ANC (South Africa)*,Import,Missiles/Rockets/Bombs,0.000505,0.00,0.24,0.0,0.000000,0.000002,0.0,0.000
1,ANC (South Africa)*,Export,Missiles/Rockets/Bombs,0.000000,0.00,0.24,0.0,0.000000,0.000002,0.0,0.000
2,ANC (South Africa)*,Overall,Missiles/Rockets/Bombs,0.001000,0.00,0.24,0.0,0.000000,0.000002,0.0,0.000
3,Afghanistan,Import,Aircraft,3.161037,0.00,15079.26,0.0,0.000000,0.107573,0.0,4.626
4,Afghanistan,Export,Aircraft,0.000000,0.00,15079.26,0.0,0.000000,0.107573,0.0,4.626
...,...,...,...,...,...,...,...,...,...,...,...
4672,unknown supplier(s),Export,Naval Vessels,0.018716,1260.08,0.00,0.0,0.001675,0.000000,0.0,0.095
4673,unknown supplier(s),Overall,Naval Vessels,0.019000,1260.08,0.00,0.0,0.001675,0.000000,0.0,0.095
4674,unknown supplier(s),Import,ZZ_loop_score_Aircraft,0.000000,1260.08,0.00,0.0,0.001675,0.000000,0.0,0.095
4675,unknown supplier(s),Export,ZZ_loop_score_Aircraft,0.053452,1260.08,0.00,0.0,0.001675,0.000000,0.0,0.095


In [160]:
# Melt the result DataFrame
melted_result = result_with_loop.reset_index().melt(
    id_vars=['Country'],  # Columns to keep as identifiers
    var_name='Metric Name',  # Name for the new column with metric names
    value_name='Metric Value'  # Name for the new column with metric values
).sort_values(by=['Country', 'Metric Name'])

# Display the melted DataFrame
display(melted_result)
melted_result.to_csv('final_data/'+current_date+'_country_score_data_melted.csv')

,Country,Metric Name,Metric Value
2088,ANC (South Africa)*,Aircraft_Normalized_Exports,NaN
2610,ANC (South Africa)*,Aircraft_Normalized_Gross,NaN
2349,ANC (South Africa)*,Aircraft_Normalized_Imports,NaN
1827,ANC (South Africa)*,Aircraft_Score,NaN
1566,ANC (South Africa)*,Country_Score,0.000000
...,...,...,...
6785,unknown supplier(s),Unmanned Vehicles_Normalized_Imports,0.000000
6263,unknown supplier(s),Unmanned Vehicles_Score,0.212000
10700,unknown supplier(s),ZZ_loop_export_score,0.093776
10961,unknown supplier(s),ZZ_loop_import_score,0.000000
